# A simple example III 

### Installation instruction

It is recommended that you clone the qthought repository to your local machine and then run

in the qthought folder. If you did not pip install qthought, you can use the following quick-fix by uncommenting and adapting to your local file path

In [ ]:
# import sys
# import os
# to run the example, set the following path to the folder path of qthought on your machine
# sys.path.append(os.path.abspath('/Users/nuri/qthought/qthought')) 

### Start of the example

This is a detailed explanation of how to write a simple protocol for two parties thought experiment. Prior to reading this, it is recommended to take a look at the PDF description file *simple_examples*.

First, we import the ProjectQ operations needed for the protocol. To initialize the state of the qubit Alice and Bob measure sequentially, we need the Hadamard gate.
We also import *Protocol* and *ProtocolStep* classes to be able to define steps of the protocol; *QuantumSystem* to operate quantum systems of different dimensionality; *Agent* class and all functions from the *copenhagen_theory* module.
We don't import *consistency* rules here, as in this example agents don't have to look at each other's viewpoints and glue them together. For an example where they do, please refer to the Frauchiger-Renner thought experiment protocol.

In [50]:
import qthought.utils as ut
from qthought.protocol import Protocol, ProtocolStep
from qthought.quantumsystem import QuantumSystem
from qthought.agents import Agent
from qthought.interpretations.copenhagen_theory import *
from projectq.ops import H, Measure

The first action of the protocol (at time $t=0$) is the initilization of a qubit *s* in a $\frac{1}{\sqrt{2}} (|0> + |1>)$ state. After defining the action, we define the step of the protocol by specifying: domain of action; written description of the action, which will be used for printouts during the run; time of the step; and which action variable being described.

In [38]:
# Step 1: The qubit R is prepared
# ----------------------------------------------------------
def action1(qsys:QuantumSystem):
    H | qsys['r']
    
step1 = ProtocolStep(domain={'Qubit': ['r']},
                     descr = 'Prepare Qubit R by applying H ',
                     time  = 0,
                     action = action1)

In the second action ($t=1$) Alice measures *R* and writes down the result to her memory. Here the *observe* function comes in play: first variable corresponds to the memory (*Alice_memory*), and the second variable - to the system being measured (*alice_qubit*). In the *step2* variable, we again specify the step by its domain - which now includes an object *Alice* of the class *Agent* with 1 memory qubit.

In [39]:
# Step 2: Alice measures the qubit R
# ----------------------------------------------------------
def action2(qsys:QuantumSystem):
    observe(qsys['Alice_memory'],qsys['r'])
    
step2 = ProtocolStep(domain={'Qubit': ['r'],
                             'AgentMemory(1)': ['Alice']},
                     descr = 'Alice observes R',
                     time  = 1,
                     action = action2)

At $t=2$, Alice makes an inference (which one, we will define later).

In [40]:
# Step 3: Alice makes inference
# ----------------------------------------------------------
def action3(qsys):
    qsys['Alice'].make_inference()


step3 = ProtocolStep(domain={'Agent(1,1)': ['Alice']},
                     descr='Alice makes an inference',
                     time=2,
                     action=action3)

At $t=3$, her reasoning and her measurement is reversed by Bob.

In [41]:
# Step 4: Reverse inference making in Alice
# ----------------------------------------------------------
def action4(qsys):
    qsys['Alice'].make_inference(reverse=True)
    observe(qsys['Alice_memory'], qsys['r'], reverse=True)


step4 = ProtocolStep(domain={'Agent(1,1)': ['Alice']},
                      descr='Reverse Alice reasoning',
                      time=3,
                      action=action4)

At $t=4$, Bob measures Alice's memory.

In [42]:
# Step 6: Bob measures Alice's memory qubit
# ----------------------------------------------------------
def action5(qsys:QuantumSystem):
    observe(qsys['Bob_memory'],qsys['Alice_memory'])
    
step5 = ProtocolStep(domain={'AgentMemory(1)': ['Alice'],
                             'AgentMemory(1)': ['Bob']},
                     descr = 'Bob observes Alice',
                     time  = 4,
                     action = action5)

Now we define the protocol as sum of its steps.

In [43]:
p_steps = [step1, step2, step3, step4, step5]
p = sum(p_steps)
p 

Step 0: Prepare Qubit R by applying H (t:0)
Step 1: Alice observes R(t:1)
Step 2: Alice makes an inference(t:2)
Step 3: Reverse Alice reasoning(t:3)
Step 4: Bob observes Alice(t:4)

Requirements: 
------------------------------
Qubit             ['r']
AgentMemory(1)    ['Bob']
Agent(1,1)        ['Alice']

We initialize the quantum system by obtaining the requirements from the protocol *p*.

In [58]:
qsys = QuantumSystem(p.get_requirements())

(Note: This is the (slow) Python simulator.)
Require Qubit r
Require AgentMemory(1) Bob
Require Agent(1,1) Alice


Exception ignored in: <function Qubit.__del__ at 0x7fc661377d90>
Traceback (most recent call last):
  File "/Users/nuri/.local/lib/python3.7/site-packages/projectq/types/_qubit.py", line 135, in __del__
    self.engine.deallocate_qubit(weak_copy)
  File "/Users/nuri/.local/lib/python3.7/site-packages/projectq/cengines/_basics.py", line 153, in deallocate_qubit
    tags=[DirtyQubitTag()] if is_dirty else [])])
  File "/Users/nuri/.local/lib/python3.7/site-packages/projectq/cengines/_main.py", line 288, in send
    raise compact_exception  # use verbose=True for more info
RuntimeError: Qubit has not been measured / uncomputed. Cannot access its classical value and/or deallocate a qubit in superposition!
 raised in:
'  File "/Users/nuri/.local/lib/python3.7/site-packages/projectq/backends/_sim/_pysim.py", line 139, in get_classical_value'
'    raise RuntimeError("Qubit has not been measured / "'
Exception ignored in: <function Qubit.__del__ at 0x7fc661377d90>
Traceback (most recent call l

Alice's inference is about the state of Bob's memory after he measures hers at the time $t=4$. In this protocol, we only use the partial inference for Alice's prediction: she only infers what happens in the case when her memory starts in the state |0> (which is the default initial state of the memory). The funnction then takes form *forward_inference_partial(protocol, 'A_memory', a=0, t_A, 'B_memory', t_B)*.

In [56]:
TA = forward_inference_partial(p, 'Alice_memory', 0, 1, 'Bob_memory', 4)

(Note: This is the (slow) Python simulator.)


/Users/nuri/anaconda3/lib/python3.7/site-packages/qthought/agents.py:214: UserWarning: make_inference called without setting an inference_table
  warn('make_inference called without setting an inference_table')
/Users/nuri/anaconda3/lib/python3.7/site-packages/qthought/agents.py:214: UserWarning: make_inference called without setting an inference_table
  warn('make_inference called without setting an inference_table')


We prepare the inference table of Alice.

In [47]:
no_prediction_state = 1
qsys['Alice'].set_inference_table(TA, no_prediction_state)
qsys['Alice'].prep_inference() 

Finally, we run the protocol.

In [48]:
p.run(qsys)

0 Prepare Qubit R by applying H  t:0
State:
0.71|000000> + 0.71|000001>
1 Alice observes R t:1
State:
0.71|000000> + 0.71|000101>
2 Alice makes an inference t:2
State:
0.71|000000> + 0.71|000101>
3 Reverse Alice reasoning t:3
State:
0.71|000000> + 0.71|000001>
4 Bob observes Alice t:4
State:
0.71|000000> + 0.71|000001>


We measure Bob's memory,

In [53]:
print('Measure Bobs memory:')
Measure | qsys['Bob_memory']
qsys.print_wavefunction()
b = int(qsys.readout('Bob_memory'))
print('b =',b)

Measure Bobs memory:
0.71|000000> + 0.71|000001>
b = 0


and compare it with Alice's prediction which she made at time $t=1$.

In [57]:
TA

In:(Alice_memory:t1)  |  Out: (Bob_memory:t4)
----------------------------------------------------
           0          |        [0]